# Auth

> The easiest way to add Google OAuth authentication to your Plash apps

## Why Use Plash Auth?

Setting up Google OAuth authentication traditionally requires:

- Google Cloud Console project setup and OAuth consent screen configuration
- Secure credential management and rotation in production
- Managing redirect URLs across development, staging, and production environments
- Complex local testing workarounds (OAuth typically breaks without HTTPS and registered domains)

**Plash Auth eliminates this complexity** by providing a simple wrapper around the OAuth flow. We handle all the Google Cloud setup, credential management, and redirect configuration for you.

## Tutorial

### 0. Setup

This tutorial will show you how to add Google OAuth authentication to your FastHTML apps deployed on Plash. With Plash's built-in auth system, you can easily implement secure sign-in functionality without managing OAuth secrets or redirect URLs yourself.

**Prerequisites:**

* A registered account at [https://pla.sh](https://pla.sh)
* Completed the [basic tutorial](../index.qmd) for deploying your first app

In this tutorial we'll focus on FastHTML. But any Plash app can technically make use of Plash Auth.

### 1. Create Your Auth App

First create a new directory for our auth example.

```bash
cd auth-example-app
```

In [ ]:
#| hide
%cd ../../examples/auth/

/Users/rensdimmendaal/git/repos/plash_cli/examples/auth


### 2. Create your app

Create a `main.py` file for your app and paste in the minimum working example from below:

In [ ]:
%%writefile main.py
from fasthtml.common import *
from plash_cli.auth import *

app, rt = fast_app()

@rt
def index(session):
    if uid:=session.get('uid'):   # <1>
            return (H1(f"Welcome! You are logged in as user: {uid}"), 
                    A("Logout", href="/logout"))
    else: 
        return (
            H1("Welcome! Please sign."), 
            A("Sign in with Google", href=mk_signin_url(session)))  # <2>

@rt(signin_completed_rt)  # <3>
def signin_completed(session, signin_reply: str):
    try: 
        uid = goog_id_from_signin_reply(session, signin_reply)  # <4>
        session['uid'] = uid
        return RedirectResponse('/', status_code=303)
    except PlashAuthError as e:  # <5> 
        return Div(
            H2("Login Failed"),
            P(f"There was an error signing you in: {e}"),
            A("Try Again", href="/")
        )
    
@rt('/logout')
def logout(session):
    session.pop('uid')  # <6>
    return RedirectResponse('/', status_code=303)

serve()


Overwriting main.py


1. Verify if user is logged in
2. Generate Auth login URL
3. Receive Auth callback
4. Extract user ID from succesful Auth response
5. Handle Auth authentication errors
6. Clear user session to log out

### 3. Add requirements

Create your `requirements.txt` file with the necessary packages:

In [ ]:
%%writefile requirements.txt
python-fasthtml
git+https://github.com/AnswerDotAI/plash_cli.git@feat-add-oauth

Overwriting requirements.txt


### 4. Deploy Your Auth App

With those two files created. Now we are ready to deploy.

```bash
plash_deploy
```

### 5. Try it out!

Visit your deployed app:

```bash
plash_view
```

Test the authentication flow:

1. **Sign in** → redirects to Google OAuth
2. **Grant permission** → returns to your app with user ID
3. **Session management** → handled by FastHTML sessions
4. **Logout** → clears session

## Limitations

### Local Development

Plash Auth only works when deployed on Plash. When you run locally with `python main.py`, you'll get a test user with ID `424242424242424242424` for development.

If you need realistic authentication testing, deploy a development version (e.g. `dev-my-app.pla.sh`) since Plash deployments are fast and low-cost.

### User Data Access

Plash Auth only provides the user's unique Google ID. If you need additional user information (name, email) or Google service access (Drive, Gmail), you'll need to implement full OAuth yourself using [FastHTML's OAuth documentation](https://www.fastht.ml/docs/explains/oauth.html).

**For most applications that just need secure user authentication, Plash Auth is the simplest solution.**